[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/J-Quants/jquants-api-client-python/blob/master/examples/20230126-006-mkdeco-jquants-kaggle-8th.ipynb)

本ノートブックは J-Quants JPX Tokyo Stock Exchange Prediction の8位入賞モデルを J-Quants API を使用して動かしてみるサンプルノートブックとなります。


J-Quants API を使用するために元のコードから一部改変しています。本ノートブックはあくまでもサンプルとしての提供を目的としており、予測精度については十分な評価・検証を実施しておりません。


* 本ノートブックはGoogleドライブにデータを書き込みます。
* 本ノートブックはデータの取得に J-Quants API を使用します。


■注意事項

本ノートブックは投資勧誘を目的としたものではありません。投資に関するご決定はご自身のご判断において行われるようお願いいたします。
掲載された情報に誤りがあった場合や、第三者によるデータの改ざん、データのダウンロード等によって生じた障害等に関し、本コミュニティは事由の如何を問わず一切責任を負うものではありませんので、あらかじめご了承ください。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# データを保存するディレクトリを指定します。Google Driveに保存します。
DATA_DIR = "/content/drive/MyDrive/drive_ws/mkdeco"
# J-Quants JPX Tokyo Stock Exchange Prediction の入賞モデルのリポジトリを取得して8位モデルのコードを取得します
REPO_DIR = "jquants-kaggle"
MODEL_DIR = "winner-models/8th"
MODEL_PATH = f"{REPO_DIR}/{MODEL_DIR}"

In [ ]:
! git clone https://github.com/J-Quants/JPXTokyoStockExchangePrediction.git $REPO_DIR

In [ ]:
import sys
sys.path.append(f"{MODEL_PATH}")

In [ ]:
# データの取得に J-Quants API を使用します。
! python3 -m pip -q install jquants-api-client

In [ ]:
# lightGBMのバージョンを調整します。
! python3 -m pip -q install -U lightgbm==3.3.5

In [ ]:
import os
import sys
import pickle

import numpy as np
import pandas as pd
import lightgbm
from tqdm.auto import tqdm

import jquantsapi as jqapi

# 上記で取得した独自ライブラリ
import Features
from Preprocessing import StockDataPreprocessor
from Trackers import StockTracker, FeatureType

In [ ]:
# J-Quants APIを使用してデータを取得します
# /content/drive/MyDrive/drive_ws/secret/jquants-api.toml にuser_idとパスワードを記載した設定ファイルを置いてあります
# ref. https://github.com/J-Quants/jquants-api-client-python#%E8%A8%AD%E5%AE%9A%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E4%BE%8B
cli = jqapi.Client()

In [ ]:
# 予測対象銘柄を指定するために銘柄一覧を取得します
df_list = cli.get_list()

In [ ]:
df_list.tail(2)

In [ ]:
# 株価情報を取得します
LAST_N_BDAY = 100
START_DT = (pd.Timestamp.now() - pd.offsets.BDay(LAST_N_BDAY)).strftime("%Y-%m-%d")
END_DT = pd.Timestamp.now().strftime("%Y-%m-%d")
PRICE_FILE = f"{DATA_DIR}/20230113-mkdeco-jquants-kaggle-8th-price-{START_DT}_{END_DT}.parquet"
if not os.path.isfile(PRICE_FILE):
    os.makedirs(DATA_DIR, exist_ok=True)
    df_price = cli.get_price_range(start_dt=START_DT, end_dt=END_DT)
    df_price.to_parquet(PRICE_FILE)
df_price = pd.read_parquet(PRICE_FILE)

In [ ]:
df_price.tail(2)

In [ ]:
df_list["ScaleCategory"].unique()

In [ ]:
# 対象をTOPIX500銘柄に絞り込みます。
topix500_list = sorted(df_list.loc[df_list["ScaleCategory"].isin(["TOPIX Core30", "TOPIX Large70", "TOPIX Mid400"]), "Code"].unique())
df_price = df_price.loc[df_price["Code"].isin(topix500_list)]
df_price["Code"].unique().shape

In [ ]:
# データを調整します。
df_price_kaggle = df_price.sort_values(["Code", "Date"])
# 銘柄コードを4桁にします
df_price_kaggle["SecuritiesCode"] = df_price_kaggle["Code"].str[:4].astype(np.int16)
# Kaggleデータに存在したカラムを追加します
df_price_kaggle["ExpectedDividend"] = 0.0
df_price_kaggle["SupervisionFlag"] = False
df_price_kaggle["Target"] = np.nan
df_price_kaggle["Target_open2close"] = np.nan

def adjust(df):
    # 調整係数を1日分ずらします
    df["AdjustmentFactor"] = df["AdjustmentFactor"].shift(-1)
    # Target列を作成します
    df["Target"] = df["AdjustmentClose"].pct_change().shift(-2)
    # target_pen2close列を作成します
    df["Target_open2close"] = df[["AdjustmentOpen", "AdjustmentClose"]].pct_change(axis=1)["AdjustmentClose"].shift(-1)
    return df

df_price_kaggle = df_price_kaggle.groupby("Code").apply(adjust).reset_index(drop=True)
# 予測対象の銘柄コード(4桁)を保存しておきます。
target_codes = sorted(df_price_kaggle["SecuritiesCode"].unique())

In [ ]:
df_price_kaggle.tail(2)

In [ ]:
# 公開されている予測モデルを取得します
MODEL_FILE = "lgbm.pickle"
with open(f"{MODEL_PATH}/{MODEL_FILE}", mode="rb") as f:
    model = pickle.load(f)

In [ ]:
# モデルのパラメータを確認します
model.get_params()

In [ ]:
# 特徴量などを設定します

In [ ]:
features = [Features.Amplitude(), Features.OpenCloseReturn(), Features.Return(), 
            Features.Volatility(10), Features.Volatility(30), Features.Volatility(50), 
            Features.SMA("Close", 3), Features.SMA("Close", 5), Features.SMA("Close", 10),
            Features.SMA("Close", 30),
            Features.SMA("Return", 3), Features.SMA("Return", 5), 
            Features.SMA("Return", 10), Features.SMA("Return", 30),
           ]

In [ ]:
## Specify training columns and specify categorical columns
training_cols = ['SecuritiesCode', 'Open', 'High', 'Low', 'Close',
                 'Volume', 'AdjustmentFactor', 'ExpectedDividend', 
                 'SupervisionFlag']

for feature in features:
    training_cols.append(feature.name)

categorical_cols = ["SecuritiesCode", "SupervisionFlag"]
target_col = ["Target"]

In [ ]:
class Algo:
    
    def __init__(self, model, state_tracker):
        self.model = model
        self.st = state_tracker
        self.cols = ['SecuritiesCode', 'Open', 'High', 'Low', 'Close',
                 'Volume', 'AdjustmentFactor', 'ExpectedDividend', 
                 'SupervisionFlag']

        # 元のコードではstock_idがハードコーディングされているため一部修正します。
        for feature in self.st.local_features[list(self.st.local_features)[0]]:
            self.cols.append(feature.name)

    def add_rank1(self, df):
        # ランク付けをシンプルにしています。
        df["Rank"] = len(df) - df["Prediction"].rank(method="first")
        return df
    
    def predict_online(self, prices):
        prices = self.st.online_update_apply(prices)[self.cols]
        if not prices["SecuritiesCode"].is_monotonic_increasing:
            prices = prices.sort_values(by="SecuritiesCode")
        prices["Prediction"] = self.model.predict(prices)
        return self.add_rank1(prices)

In [ ]:
class StateTracker:

    def __init__(self, features, stock_ids):
        # 元のコードではstock_idがハードコーディングされているため一部修正します。
        self.stock_ids = stock_ids
        self.stock_trackers = {}
        for s_id in self.stock_ids:
            self.stock_trackers[s_id] = StockTracker(s_id)
        self.global_features = []

        self.local_features = {}

        for feature in features:

            if feature.feature_type == FeatureType.GLOBAL:
                self.global_features.append(feature)
            elif feature.feature_type == FeatureType.LOCAL:
                for stock_id in self.stock_ids:
                    if stock_id not in self.local_features:
                        self.local_features[stock_id] = []
                    self.local_features[stock_id].append(feature.copy())

    def update_single_row(self, row):
        stock_id = row["SecuritiesCode"]
        row, status_code = self.stock_trackers[stock_id].update(row)
        row["StatusCode"] = status_code
        for feature in self.local_features[stock_id]:
            row = feature.update_row(row)
        return row

    def online_update_apply(self, prices):
        return prices.apply(lambda row: self.update_single_row(row), axis=1)


In [ ]:
# 推論します (初期設定のデータ期間 (100日) の場合、10分程度かかります)
st = StateTracker(features, target_codes)
algo = Algo(model, st)
# 取得するランキングの上下N銘柄数を指定します
# EVAL_N = 10

buff = []
for grouped_prices in tqdm(df_price_kaggle.groupby("Date")):
    prices = grouped_prices[1].copy()
    if not prices["SecuritiesCode"].is_monotonic_increasing:
        prices = prices.sort_values("SecuritiesCode")
    
    prices["Rank"] = algo.predict_online(prices)["Rank"]  # make your predictions here
    prices = prices.sort_values("Rank")
    # ランキングの上下N銘柄のみ取得します
    # prices = pd.concat([prices.iloc[:EVAL_N], prices.iloc[-EVAL_N:]])

    buff.append(prices)   # register your predictions

In [ ]:
df_ret = pd.concat(buff)

In [ ]:
df_ret.head(2)

In [ ]:
df_ret.tail(2)

In [ ]:
# 日次の順位相関を確認します
df_ret.groupby("Date")[["Target", "Rank"]].corr(method="spearman").xs("Target", level=1)["Rank"].describe()

In [ ]:
# 日次の順位相関の累積をプロットします
df_ret.groupby("Date")[["Target", "Rank"]].corr(method="spearman").xs("Target", level=1)["Rank"].cumsum().plot(figsize=(20, 8))